# RAG Az Azure AI Agent Service és a Semantic Kernel használatával

Ez a kódrészlet bemutatja, hogyan hozhatunk létre és kezelhetünk egy Azure AI ügynököt a visszakeresés-alapú generálás (RAG) céljából az `Azure AI Agent Service` és a `Semantic Kernel` segítségével. Az ügynök a lekérdezett kontextus alapján dolgozza fel a felhasználói kérdéseket, és ennek megfelelően pontos válaszokat ad.


SQLite verzió javítása  
Ha a következő hibával találkozol:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Kommenteld ki ezt a kódrészletet a jegyzetfüzeted elején:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Csomagok importálása
Az alábbi kód importálja a szükséges csomagokat:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Visszakeresés-alapú Generálás Szemantikus Kernel és Azure AI Agent Service segítségével

Ez a példa bemutatja, hogyan használható az **Azure AI Agent Service** a **Visszakeresés-alapú Generálás (RAG)** megvalósítására, egy nyelvi modell és egy feltöltött dokumentumból származó, specifikus kontextus kombinálásával.

### Hogyan működik

1. **Dokumentum feltöltése**: Egy markdown fájl (document.md), amely információkat tartalmaz (Contoso utazási biztosítási szabályzata), feltöltésre kerül az agent szolgáltatásba.

2. **Vektortár létrehozása**: A dokumentum indexelésre kerül egy vektortárban, hogy lehetővé tegye a szemantikus keresést a tartalmában.

3. **Agent konfigurálása**: Egy agent kerül létrehozásra a `gpt-4o` modell használatával, az alábbi szigorú utasításokkal:
   - Csak a dokumentumból visszakeresett tartalom alapján válaszoljon a kérdésekre.
   - Tagadja meg a válaszadást, ha a kérdés kívül esik a dokumentum tartalmán.

4. **File Search Tool integrációja**: A `FileSearchTool` regisztrálva van az agenthez, lehetővé téve a modell számára, hogy kereséseket végezzen és releváns részleteket hívjon elő az indexelt dokumentumból az inferencia során.

5. **Felhasználói interakció**: A felhasználók kérdéseket tehetnek fel. Ha releváns információ található a dokumentumban, az agent egy megalapozott választ generál.  
   Ha nem, az agent egyértelműen közli, hogy a dokumentum nem tartalmaz elegendő információt.


### Fő Funkció



Győződjön meg arról, hogy először futtatja az `az login` parancsot az Azure CLI használatával, hogy a megfelelő hitelesítési kontextus biztosítva legyen a `DefaultAzureCredential` használata közben. Az Azure AI Agent Service nem használ API kulcsokat.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Rendben, kérem adja meg a fordítandó szöveget.



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
